# SageMaker JumpStart Fine-Tune ハンズオン

## 目次
1. [概要](#anchor1)
1. [準備](#anchor2)
1. [Amazon SageMaker Ground Truthを用いたラベリング](#anchor3)
1. [Fine-Tuneの準備](#anchor4)
1. [Fine-Tuneの実行](#anchor5)
1. [デプロイ](#anchor6)
1. [推論](#anchor7)
1. [リソースの削除](#anchor8)

<a id="anchor1"></a>
## 1. 概要
Amazon SageMaker JumpStartは、機械学習をすばやく簡単に開始するのに役立ちます。機械学習ワークロードを容易に開始するために、SageMaker JumpStartは最も一般的なユースケース向けの一連のソリューションを提供し、数回クリックするだけで簡単にデプロイできます。この学習モデルに独自のデータセットを加えたい場合、Fine-Tuneを行うことができます。このプロセスは転移学習とも呼ばれ、小さなデータセット・短いトレーニング時間でより正確なモデルを作成することができます。本ハンズオンでは、まずSageMaker Ground Truthを用いて学習モデルに加えたい独自のデータセットにラベル付けを行います。その後、予め用意された物体検出の学習モデルに独自のデータセットを加えてFine-Tuneを行います。

---
<a id="anchor2"></a>
## 2. 準備
### 2.1. S3バケットの作成
本ハンズオンで扱うAmazon SageMaker Ground Truth, Amazon SageMaker JumpStart Fine-Tuneでは、データのやり取りをS3バケットを用いて行います。本節では以降の作業で必要なS3バケットの作成を行います。


In [ ]:
import boto3
import uuid
S3_BUCKET = 'jumpstart-finetune-handson' + str(uuid.uuid4()) # 一意の文字列にするためUUIDを付与
region = boto3.session.Session().region_name
location = {'LocationConstraint': region}
s3_client = boto3.client('s3')
response = s3_client.create_bucket(Bucket=S3_BUCKET,CreateBucketConfiguration=location)


次に、S3バケットに必要なフォルダを作成します。  
作成するS3バケットのディレクトリ構成は以下となります。
+ /S3Bucket
    + /bounding_box
        + /ground_truth
        + /training
            + /images
    + /test        

作成するS3バケットのディレクトリは、それぞれ以下の用途で用いられます。

__/bounding_box__  
+ Ground Truthの入力・出力データを格納するディレクトリです。/ground_truthに出力データを、/training/imagesにラベル付けするデータを格納します。  

__/test__  
+ SageMaker JumpStartの学習モデルをFine-tuneした後、推論に用いるデータを格納します。

In [ ]:
import os
BOUNDING_BOX_PREFIX = 'bounding_box'
GROUND_TRUTH_PREFIX = os.path.join(BOUNDING_BOX_PREFIX, 'ground_truth')
BB_TRAINING_PREFIX =os.path.join(BOUNDING_BOX_PREFIX, 'training')
INPUT_IMAGES_PREFIX = os.path.join(BB_TRAINING_PREFIX, 'images')
TEST_PREFIX = 'test'

res = s3_client.put_object(Bucket=S3_BUCKET, Key=(BOUNDING_BOX_PREFIX+'/'))
res = s3_client.put_object(Bucket=S3_BUCKET, Key=(GROUND_TRUTH_PREFIX+'/'))
res = s3_client.put_object(Bucket=S3_BUCKET, Key=(BB_TRAINING_PREFIX+'/'))
res = s3_client.put_object(Bucket=S3_BUCKET, Key=(INPUT_IMAGES_PREFIX+'/'))
res = s3_client.put_object(Bucket=S3_BUCKET, Key=(TEST_PREFIX+'/'))


### 2.2. 画像データのアップロード
前節で作成したS3バケットに、画像データをアップロードします。まずは、tar.gzファイルを解凍します。

In [ ]:
!tar -zxvf images.tar.gz --no-same-owner

以下の手順で、画像データをS3バケットにアップロードします。本画像データはGround Truthでのラベリング、SageMaker JumpStartのFine-Tuneに用います。

In [ ]:
# images/train下の.jpgの一覧を取得し、S3にアップロード
train_dir = os.path.join('images','train')
images = os.listdir(path=train_dir)

for image in images:
    destination = os.path.join(INPUT_IMAGES_PREFIX,image)
    s3_client.upload_file(os.path.join(train_dir,image), S3_BUCKET, destination)

---
<a id="anchor3"></a>
## 3. Amazon SageMaker Ground Truthを用いたラベリング
本章ではAmazon SageMaker Ground Truthを用いたラベリング作業を行います。**ラベリング作業を実施せず、SageMaker JumpStart Fine-Tuneのみを試したい場合は、本章は飛ばして4章の[Fine-Tuneの準備](#anchor4)へ進んで下さい。**  

Amazon SageMaker Ground Truth はフルマネージド型のデータラベル付けサービスで、機械学習のための高精度なトレーニングデータセットを簡単に構築することができます。カスタム、または組み込み済みのデータラベル付けワークフローを使用して、SageMaker Ground Truth コンソールから数分でデータのラベル付けを開始することができます。本章では、Fine-Tuneさせる画像データをGround Truthでラベル付けします。

<a id="anchor3.1"></a>
### 3.1. Manifestファイルの準備
本節ではGround Truthで必要なManifestファイルを作成した後、S3にアップロードします。Manifestファイルはラベリングジョブの入力データセットとして使用されます。詳細は、[ドキュメント](https://docs.aws.amazon.com/ja_jp/sagemaker/latest/dg/sms-input-data-input-manifest.html)を参照下さい。

In [ ]:
import json

def create_manifest(s3_bucket, image_path):
    '''
    Ground Truthでのラベリングに必要なmanifestファイルを生成します。
    
    Parameters
    ----------
    s3_bucket : str
        manifestファイルを作成する画像データが格納されているS3Bucket名
        
    image_path : str
        manifestファイルを作成する画像データのパス

    Returns:
    -------
    manifest_file : str
        作成したmanifestファイル名
    '''
    img_files = get_file_list_from_s3(s3_bucket, image_path)

    TOKEN = 'source-ref'
    manifest_file = os.path.join('data', 'manifest.json')
    if not (os.path.exists('data')):
        os.mkdir('data')
    with open(manifest_file, 'w') as fout:
        for img_file in img_files:
            fname = f's3://{s3_bucket}/{img_file.key}'
            fout.write(f'{{"{TOKEN}": "{fname}"}}\n')

    return manifest_file

def get_file_list_from_s3(s3_bucket, image_path):
    '''
    S3に格納されている画像ファイルパスリストを取得する
    
    Parameters
    ----------
    s3_bucket : str
        manifestファイルを作成する画像データが格納されているS3Bucket名
        
    image_path : str
        manifestファイルを作成する画像データのパス

    Returns:
    -------
    img_files : list
        S3に格納されている画像ファイルパスリスト
    '''
    s3_rec = boto3.resource('s3')
    bucket = s3_rec.Bucket(s3_bucket)
    objs = list(bucket.objects.filter(Prefix=image_path))
    img_files = objs[1:]  # first item is the folder name
    n_imgs = len(img_files)
    print(f'there are {n_imgs} images \n')
    return img_files
    

def upload_manifest(s3_bucket, prefix, manifest_file):
    '''
    manifestファイルをS3へアップロードします。

    Parameters
    ----------
    s3_bucket : str
        manifestファイルを格納するS3Bucket名
        
    prefix : str
        manifestファイルを格納するprefix名
        
    manifest_file : str
        manifestファイル名

    '''

    s3_rec = boto3.resource('s3')
    destination = os.path.join(prefix, 'manifest.json')
    print(f'uploading manifest file to {destination} \n')
    s3_rec.meta.client.upload_file(manifest_file, s3_bucket, destination)

man_file = create_manifest(S3_BUCKET, INPUT_IMAGES_PREFIX)
upload_manifest(S3_BUCKET, BOUNDING_BOX_PREFIX, man_file)


### 3.2. ワーカーの登録
本ハンズオンではSageMaker Ground Truthで画像データをラベリングするために、ワーカーとしてプライベートチームを作成します(プライベートチーム以外にもワーカーとして選択することが可能です。その他のワーカーの選択肢については[SageMakerのドキュメント](https://docs.aws.amazon.com/ja_jp/sagemaker/latest/dg/sms-getting-started-step3.html)を参照してください。)。本節ではプライベートチームを作成します。

1. **[SageMaker](https://console.aws.amazon.com/sagemaker/home)**のコンソール画面へアクセスし、左側のナビゲーションから**Ground Truth**を展開し、**ラベリングワークフォース**を選択します。
1. 画面上部の**プライベート**を選択し、**プライベートチームを作成**をクリックします。
1. 任意の**チーム名**を入力し、**ワーカーの追加**で**新しいAmazon Cognitoユーザーグループを作成する**を選択、最後に**プライベートチームを作成**をクリックします。
1. **ワーカー**欄にある**新しいワーカーを招待**をクリックします。
1. 今回のハンズオンでラベリングを実施するワーカーのメールアドレスを記載します。アドレスを複数登録する際、カンマ区切りで入力して下さい。
1. 上記で作成した**プライベートチーム**を押下した後、**ワーカー**を選択し、**チームにワーカーを追加**を押下します。
1. 手順3で作成した**プライベートチーム**名を選択した後、**ワーカー**を押下します。その後、**チームにワーカーを追加**を選択し、ワーカー一覧から今回のラベリング作業を実施させたいワーカーを選択します。

以上でワーカーの登録は終了です。


### 3.3 ラベリングジョブの作成
1. **[SageMaker](https://console.aws.amazon.com/sagemaker/home)**のコンソール画面へアクセスし、左側のナビゲーションから**Ground Truth**を展開し、**ラベリングジョブ**を選択します。
1. 画面上部にある**ラベリングジョブの作成**を押下します。
1. **ジョブ名**に**jumpstart-handson-labeljob**と記入します。**入力データのセットアップ**では**手動によるデータのセットアップ**を選択します。
1. **入力データセットの場所**を設定します。**S3の参照**を押下し、本ハンズオンで作成したS3バケットを指定し、[3.1 Manifestファイルの準備](#anchor3.1)で作成したmanifestファイル（bounding_box/manifest.json）を指定します。**出力データセットの場所**は[2. S3バケットの作成](#anchor2)で作成したground_truthディレクトリ(bounding_box/ground_truth)を指定します。
1. **IAMロール**でリストを展開し、**Create a new role**を選択します。**特定のS3バケット**を選択し、[2. S3バケットの作成](#anchor2)で作成した**S3バケット**名を指定し、**作成**ボタンを押下します。
1. **タスクの選択**で**境界ボックス**を選択し、**次へ**ボタンを押下します。
1. **ワーカータイプ**で**プライベート**を選択します。3.3で作成したワーカーチームを選択します。
1. **境界ボックスラベリングツール**欄にタスクの簡単な説明を入力します（例：For labeling usb cable.）。その後、**ラベル**にモデルへ追加したい画像データに必要なラベルを追加します。ラベルは最大50個まで追加することができます。本ハンズオンではusb-a, usb-b, lightningの3種類をラベルとして追加します。
1. 画面右下の**作成**ボタンを押下します。
1. ワーカーとして登録したメールアドレスに招待メールが届いていることを確認してください。メールに記載されているリンクを押下し、ユーザー名、パスワードを入力してラベリング画面へ進み、ラベリング作業を実施して下さい。（ラベリングジョブが画面一覧に表示されるまで少し時間がかかる場合があります。何も表示されていなかった場合は少し時間が経ってから再度試して下さい。）

### 3.4 Groud Truthから出力されたoutput.manifestの取得
Ground Truthでラベリングした情報が記載されているoutput.manifestをS3から取得します。本ファイルはFine-Tuneを行うために必要なannotation.jsonを作成する際に使用します。

In [ ]:
LABELING_JOB_NAME = "jumpstart-handson-labeljob" # 独自のラベリングジョブ名にした場合はここを変更してください。
OUTPUT_MANIFEST_S3_PATH = os.path.join(GROUND_TRUTH_PREFIX, LABELING_JOB_NAME, "manifests", "output", "output.manifest")
s3 = boto3.resource('s3')
bucket = s3.Bucket(S3_BUCKET)
bucket.download_file(OUTPUT_MANIFEST_S3_PATH, os.path.join('data', 'output.manifest'))

---
<a id="anchor4"></a>
## 4.Fine-Tuneの準備
本章ではモデルへ追加する独自のデータセットを、Fine-Tune可能な形式へ処理・変換します。本ハンズオンでは、Jump StartのVISIONモデルである'SSD MobileNet 1.0'を扱います。


### 4.1.Fine-Tune実施に必要なJSONファイルの作成
ここではFine-Tuneを実施するために必要なannotations.jsonファイルを作成します。Fine-Tuneを実施する各モデルの**Fine-tune the Model on a New Dataset**の記載を参考にJSONファイルを作成します。
(annotations.jsonが作成されるまで少し時間が掛かります。)

In [ ]:
# annotations.jsonを扱うクラスを定義
class AnnotationsJson:
    '''annotaions.jsonを扱うクラス
    Parameters
    ----------
    records : list
        output.manifestの各行を辞書型で読み込みリスト化したもの
    jobname : str
        Ground Truthのラベリングジョブ名

    Attributes
    ----------
    self.__data : list
        annotations.jsonに書き込むJSONデータ
    
    '''
    def __init__(self, records, jobname):
        self.__data = self.__generate_annotations_json(records, jobname)
    
    @property
    def data(self):
        return self.__data
        
    def save(self, path):
        '''
        annotaions.jsonを保存する
        
        Parameters
        ----------
        path : str
            annotations.jsonを保存するパス
       
        '''
        with open(path, 'w') as f:
            json.dump(self.__data, f)
            
    def __generate_annotations_json(self, records, jobname):
        '''
        annotions.jsonに書き込むデータを作成する。
        
        Parameters
        ----------
        records : list
            output.manifestの各行ごとのデータ
        jobname : str
            ラベリングジョブ名
        
        Return
        ------
        data : dict
            annotations.jsonに書き込むデータ
        
        '''
        data = []
        image_id = 0
        images = []
        annotations =[]
        for record in records:
            record_json = json.loads(record)
            images.append(self.__set_images(record_json, jobname, image_id))

            for class_map in record_json[jobname]['annotations']:
                annotations.append(self.__set_annotations(class_map, image_id))
            image_id += 1
        data = dict()
        data['images'] = images
        data['annotations'] = annotations
        return data
            
    def __set_images(self, record, jobname, image_id):
        '''annotations['images']の値を生成する
        Parameters
        ----------
        record : list
            output.manifestの1行分データ
        jobname : str
            Ground Truthのラベリングジョブ名
        image_id : int
            画像ID
            
        Return
        ------
        images : dict
            annotations['images']の値
        '''
        filename = record['source-ref'].split('/')[-1]
        class_map = record[jobname]
        images = dict()
        images['file_name'] = filename
        # class_map['image_size']は辞書要素一つのリストなので[0]で取り出し
        images['height'] = class_map['image_size'][0]['height']
        images['width'] = class_map['image_size'][0]['width']
        images['id'] = image_id

        return images
        
    def __set_annotations(self, classmap, image_id):
        '''annotations['annotaions']の値を生成する
        Parameters
        ----------
        classmap : dict
            ラベリングジョブで付与したannotations情報
        image_id : int
            画像ID
            
        Return
        ------
        annotations : dict
            annotations['annotaions']の値
        
        '''
        
        annotations = dict()
        annotations['image_id'] = image_id
        try:
            annotations['bbox'] = self.__calc_bbox(classmap)
        except Exception as e:
            print('iamge_id is {}'.format(image_id))
            print('classmap is {}'.format(classmap))
            raise e
        annotations['category_id'] = classmap['class_id']

        return annotations
    
    def __calc_bbox(self,bbox_gt):
        '''bounding_boxの座標を、Ground Truth用からGluonCV用へ変換する。
        Parameter
        ---------
        bbox_gt : dict
            Grount Truthで算出したbounding boxの値が格納されている辞書
            （top, left, width, height）
            
        Return
        ------
        bbox : list
            GluonCV用bounding_box
            [xmin, ymin, xmax, ymax]
        '''

        xmin = bbox_gt['left']
        ymin = bbox_gt['top']
        xmax = bbox_gt['left'] + bbox_gt['width']
        ymax = bbox_gt['top'] + bbox_gt['height']
        bbox = [xmin, ymin, xmax, ymax]

        # 座標に負の値が含まれていないか確認
        if any((x < 0 for x in bbox)):
            raise Exception('Value of bbox is invelid : {}'.format(bbox))

        return bbox

In [ ]:
import json
# annotations.jsonを作成し、S3にアップロードする
fileopen = open(os.path.join('data', 'output.manifest'))
record = fileopen.readlines()
fileopen.close()

# 3章を実施していないとLABELING_JOB_NAMEが未定義のため
try:
    aj = AnnotationsJson(record, LABELING_JOB_NAME)
except NameError:
    LABELING_JOB_NAME = "jumpstart-handson-labeljob"
    aj = AnnotationsJson(record, LABELING_JOB_NAME)

LOCAL_ANNOTAIONS_PATH = os.path.join('data', 'annotations.json')
aj.save(LOCAL_ANNOTAIONS_PATH)

# 3章を実施していないとbucketが未定義のため
try:
    bucket.upload_file(LOCAL_ANNOTAIONS_PATH, os.path.join(BB_TRAINING_PREFIX, 'annotations.json'))
except NameError:
    s3 = boto3.resource('s3')
    bucket = s3.Bucket(S3_BUCKET)
    bucket.upload_file(LOCAL_ANNOTAIONS_PATH, os.path.join(BB_TRAINING_PREFIX, 'annotations.json'))
    

---
<a id="anchor5"></a>
## 5.Fine-Tune
ここからFine-Tuneを実施していきます。以下の手順で進めて下さい。
1. 画面左側下から2番目のSageMaker JumpStart Launch Assetsアイコンを選択し、**Browse JumpStart**を押下します。
1. **Search**欄に**Object Detection**と入力し、表示された一覧の中から**SSD MobileNet 1.0**を選択します。
1. **Fine-tune Model**セクションの**Data Source**で**Find S3 bucket**を選択します。
1. **S3 bucket name**に2.1節で作成したS3バケット名を、**Dataset directory name**に**/bounding_box/training**を選択します。
1. **Deployment Configuration**で、適当な**SageMaker Training Instance**を選択します。
1. **Hyper Parameter**で、**Epoch**数を30に設定します。
1. **Train**ボタンを押下します。

---

<a id="anchor6"></a>
## 6.デプロイ
Fine-Tuneが終了したら、以下の手順でモデルをデプロイします。
1. 画面左側下から2番目のSageMaker JumpStart Launch Assetsアイコンを選択し、**Browse JumpStart**下のドロップダウンリストから**Training Jobs**を選択します。
1. 3章で実施した**Training Job**を選択します。**Deploy Model**セクションの**Deployment Configuration**を展開し、**ml.m5.large**を選択します。また、**Endpoint Name**欄に入力されている文字列をコピーしておきます。
1. **Deploy**ボタンを押下します。  

これにてデプロイが実施されます。終了するまで5 ~ 10分かかります。

---

<a id="anchor7"></a>
## 7.推論
エンドポイントのstatusが**In Service**となったら、推論を実行し結果を確認してみましょう。事前に**Endpoint Name**の文字列をコピーしておいて下さい。推論で扱う画像を以下で表示します。

In [ ]:
from IPython.core.display import HTML
import boto3
# 推論する画像をアップロード
test_dir = os.path.join('images','test')
img_jpg = os.path.join(test_dir, 'test.jpg')
destination = os.path.join(TEST_PREFIX, 'test.jpg')
s3_client.upload_file(img_jpg, S3_BUCKET, destination)

HTML('<img src="images/test/test.jpg" alt="test" style="height: 600px;"/>'
     '<figcaption>test.jpg</figcaption>')

では、上記の画像をエンドポイントへクエリし、推論結果を取得してみましょう。以下の変数**endpoint_name**にデプロイしたモデルのEndpoint Nameを代入してから、コードを実行して下さい。
（うまく物体検出がされない場合、confidenceの値を変化させるか、再度エポック数を変更してトレーニングを行う、またはGround Truthでのラベリング処理から再度やり直して下さい。）

In [ ]:
confidence = 0.7 # 信頼度

import json
def query_endpoint(input_img):
    endpoint_name = '***' #Endpoint Nameの文字列を代入
    client = boto3.client('runtime.sagemaker')
    response = client.invoke_endpoint(EndpointName=endpoint_name, ContentType='application/x-image', Body=input_img)
    model_predictions = json.loads(response['Body'].read())
    return model_predictions

with open(img_jpg, 'rb') as file: input_img = file.read()
best_results_per_input = query_endpoint(input_img)

import matplotlib.patches as patches
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
from PIL import ImageColor
colors = list(ImageColor.colormap.values())

labels = ['type-a','type-b','lightning']
image_np = np.array(Image.open(img_jpg))
plt.figure(figsize=(20,20))
ax = plt.axes()
ax.imshow(image_np)
bboxes, classes, confidences = best_results_per_input
for idx in range(len(bboxes)):
    if confidences[idx] > confidence:
        class_num = int(classes[idx])
        left, bot, right, top = bboxes[idx]
        x, w = [val * image_np.shape[1] for val in [left, right - left]]
        y, h = [val * image_np.shape[0] for val in [bot, top - bot]]
        color = colors[hash(class_num) % len(colors)]
        rect = patches.Rectangle((x, y), w, h, linewidth=3, edgecolor=color, facecolor='none')
        ax.add_patch(rect)
        ax.text(x, y, '{} {:.0f}%'.format(labels[class_num], confidences[idx]*100), bbox=dict(facecolor='white', alpha=0.5))

## 8.リソースの削除

1. 1章で作成したS3バケットを削除します。**[S3のコンソール画面](https://s3.console.aws.amazon.com/s3/)**から該当のバケットを選択し、**空にする**を押下しバケットの中身を空にした後、バケットを削除します。
2. 4章でデプロイしたエンドポイントを削除します。**Delete Endpoint**セクションの**Delete**ボタンを押下して下さい。
3. SageMaker Studioで立ち上がっているインスタンス・カーネルセッションを停止させます。画面左側上から3つ目の停止アイコンを押下し、本ハンズオンで用いたインスタンス・カーネルセッションを停止させます。